# Deep learning advanced with yolo features demo
This notebook will guide you how we did for the deep learning advanced with yolo features model.

### Check if you have Required file(under this root dir):  
    ./
        v2_mscoco_train2014_annotations.json
        v2_mscoco_val2014_annotations.json
        bad_im_files_train.npy
        bad_im_files_val.npy
        training_annotations.npy
        vqa_py3.py
        dl_advanced_model_withYOLO.py
**************************************************************************************

### 1.0 Download needed image features
#### option 1(Recommend)
Run the code below

#### option 2
Download **fasterrcnn_output.zip** to root dir, unzip it, and rename the unzipped dir: **fasterrcnn_output**

Drive link: https://drive.google.com/open?id=1RwPiZo0g6JDPOIblmg0jDpDQf_ZWvi8F 

Download **yolo_feats.zip** to root dir, unzip it, and rename the unzipped dir: **yolo_feats**

Drive link: https://drive.google.com/file/d/1MBYlua5jg-DXI23f6T9gYZnU1JFZTEGX/view?usp=sharing

In [1]:
# Define function for downloading big file from drive:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [2]:
file_id = '1RwPiZo0g6JDPOIblmg0jDpDQf_ZWvi8F'
destination = 'fasterrcnn_output.zip'
download_file_from_google_drive(file_id, destination)

In [ ]:
!unzip -qq fasterrcnn_output.zip

In [8]:
import os
examples = sum([len(files) for r, d, files in os.walk("fasterrcnn_output")])

print("all example: ", examples)

all example:  123287


In [48]:
file_id = '1MBYlua5jg-DXI23f6T9gYZnU1JFZTEGX'
destination = 'yolo_feats.zip'
download_file_from_google_drive(file_id, destination)

In [49]:
import zipfile
with zipfile.ZipFile("yolo_feats.zip","r") as zip_ref:
    zip_ref.extractall("yolo_feats")

In [1]:
import os
train = sum([len(files) for r, d, files in os.walk("yolo_feats/train2014")])
val = sum([len(files) for r, d, files in os.walk("yolo_feats/val2014")])
print("Train example: ", train)
print("Validation example: ", val)

Train example:  82783
Validation example:  40504


### 1.1 Install required packages
Next we will first download the required packeges

In [ ]:
# Install requirement package
!pip install -r requirements.txt

### 2.0 Run non_dl_baseline script
Then simply run our script to training and save your model

In [ ]:
!python dl_advanced_model_withYOLO.py

### 3.0 Check your training/validation accuracy
Run the code below to draw a accuracy plot for the training process you just done! 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

s = np.load('dl_advance_log/with_yolo/train_acc.npy', allow_pickle=True).item()
accuracies = []
epochs = list(range(1, len(s.keys())+1))
for epoch in s.keys():
#     for i in range(0, len(s[epoch]), 20):
    accuracies.append(s[epoch][-1])

plt.plot(epochs, accuracies)

v = np.load('dl_advance_log/with_yolo/val_acc.npy', allow_pickle=True).item()
val = []
for epoch in v.keys():
#     for i in range(0, len(s[epoch]), 20):
    val.append(v[epoch])
plt.plot(epochs, val)
plt.legend(['Train Accuracy', 'Val Accuracy'], loc='upper left')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()
plt.savefig('Accuracy_dl.png')